In [22]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import requests
import urllib
import json
import getpass

In [10]:
# carga capa de centros medios de población
cp = gpd.read_file('https://raw.githubusercontent.com/guillermodangelo/tesis/master/capas/centro_poblacion.gpkg')

cp.head()

,DEPTO,geometry
0,1,POINT (575497.000 6141786.000)
1,2,POINT (517717.000 6636093.000)
2,3,POINT (587353.000 6159278.000)
3,4,POINT (777711.000 6409999.000)
4,5,POINT (431654.000 6211872.000)


In [12]:
# obtiene coordenadas en EPSG 4326
lon = cp.geometry.to_crs(epsg=4326).x
lat = cp.geometry.to_crs(epsg=4326).y
cp_repro = pd.concat( [ pd.DataFrame( list(lat), columns=['lat'] ), pd.DataFrame( list(lon), columns=['lon'] ) ], axis=1, sort=False)

# concatena en un sólo string
cp_repro['latlon'] = cp_repro.apply(lambda x:'%s,%s' % (x['lat'],x['lon']),axis=1)

# concatena ambos df
df_cp = pd.concat([cp, cp_repro], axis=1, ignore_index=False)
df_cp.head()

,DEPTO,geometry,lat,lon,latlon
0,1,POINT (575497.000 6141786.000),-34.863473,-56.174030,"-34.86347311448007,-56.17403019571399"
1,2,POINT (517717.000 6636093.000),-30.407062,-56.815547,"-30.407062416732217,-56.81554727891256"
2,3,POINT (587353.000 6159278.000),-34.704806,-56.046153,"-34.70480623405611,-56.04615274218908"
3,4,POINT (777711.000 6409999.000),-32.412531,-54.047130,"-32.412530637702794,-54.04713015664623"
4,5,POINT (431654.000 6211872.000),-34.231989,-57.742109,"-34.23198949912197,-57.74210900659811"


In [17]:
# realiza el producto cartesiano para obtener todas las combinaciones posibles de localidades
df_cp_coords = df_cp[['DEPTO', 'latlon']]
df_cp_coords['key']=0

df_prod_cartesiano = df_cp_coords.merge(df_cp_coords, on='key', how='outer')
df_prod_cartesiano.drop(columns=['key'], inplace=True)
df_prod_cartesiano.rename(columns={'DEPTO_x': 'cod_ori', 'latlon_x': 'latlon_ori',
                                       'DEPTO_y': 'cod_des', 'latlon_y': 'latlon_des'}, inplace=True)

# elimina misma cod origen-destino
df_prod_cartesiano = df_prod_cartesiano[df_prod_cartesiano['cod_ori'] != df_prod_cartesiano['cod_des']]

# concatena en un código
df_prod_cartesiano['cod'] = (df_prod_cartesiano['cod_ori'].astype(str).str.zfill(2) + 
                             df_prod_cartesiano['cod_des'].astype(str).str.zfill(2)).astype('int')

print(df_prod_cartesiano.shape[0])
df_prod_cartesiano.head(5)

342


<ipython-input-17-4e6b5e23f8e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cp_coords['key']=0


,cod_ori,latlon_ori,cod_des,latlon_des,cod
1,1,"-34.86347311448007,-56.17403019571399",2,"-30.407062416732217,-56.81554727891256",102
2,1,"-34.86347311448007,-56.17403019571399",3,"-34.70480623405611,-56.04615274218908",103
3,1,"-34.86347311448007,-56.17403019571399",4,"-32.412530637702794,-54.04713015664623",104
4,1,"-34.86347311448007,-56.17403019571399",5,"-34.23198949912197,-57.74210900659811",105
5,1,"-34.86347311448007,-56.17403019571399",6,"-33.29103587835989,-56.27098299307005",106


In [18]:
# matriz de distancia entre capitales departamentales de departamentos

# define la función obtenerDistancia que usa la API Distance Matrix de Google

def obtenerDistancia(ori, dest, API_KEY):
    '''
    Esta funcion devuelve distancia en km entre un o varios par(es) de coordenadas
    geográficas, utilizando la API de Google Distance Matrix.
    '''
    APIurl = """https://maps.googleapis.com/maps/api/distancematrix/json?
                units=metric&mode=walking&origins=""" + ori +\
                "&destinations=" + dest + "&key=" + API_KEY
    #Obtener la respuesta y cargar la representacion en un string
    response = requests.get(APIurl)
    geodataString = response.text
    geodata = json.loads(geodataString)
    # si la API no devulve resultados, la distancia será 'None'
    if geodata['rows'][0]['elements'][0]['status']=='ZERO_RESULTS':
        distancia = None
    else:
        distancia = geodata['rows'][0]['elements'][0]['distance']['value']
    return distancia

In [30]:
# setea la API KEY
API_KEY = getpass.getpass()

# ordena y resetea el índice
df_prod_cartesiano.sort_values(by=['cod_ori', 'cod_des'], inplace=True)
df_prod_cartesiano.reset_index(drop=True, inplace=True)

print(df_prod_cartesiano.shape[0])
df_prod_cartesiano.head(3)

 ·······································


342


,cod_ori,latlon_ori,cod_des,latlon_des,cod
0,1,"-34.86347311448007,-56.17403019571399",2,"-30.407062416732217,-56.81554727891256",102
1,1,"-34.86347311448007,-56.17403019571399",3,"-34.70480623405611,-56.04615274218908",103
2,1,"-34.86347311448007,-56.17403019571399",4,"-32.412530637702794,-54.04713015664623",104


In [32]:
df_prod_cartesiano.shape[0]

342

In [33]:
# rpeuab con 1 caso
obtenerDistancia( df_prod_cartesiano.latlon_ori[0] , df_prod_cartesiano.latlon_des[0] , API_KEY)

610726

In [35]:
# son 342 díadas de centros de población
# for loop para obtener una lista con las distancias entre cp
# aplica la función obtenerDistancia para combinaciones de cp

distancias=[]

for x in range(0, df_prod_cartesiano.shape[0]):
  distancias.append(obtenerDistancia( df_prod_cartesiano.latlon_ori[x] , df_prod_cartesiano.latlon_des[x] , API_KEY))

In [36]:
print(len(distancias))

342


In [37]:
# pega con el df original
# se conviernte en matriz de distancias, data frame
df_distancias_google = pd.concat([df_prod_cartesiano.reset_index(drop=True), pd.DataFrame(distancias, columns=['distancia'])], axis=1)
df_distancias_google.head()

,cod_ori,latlon_ori,cod_des,latlon_des,cod,distancia
0,1,"-34.86347311448007,-56.17403019571399",2,"-30.407062416732217,-56.81554727891256",102,610726
1,1,"-34.86347311448007,-56.17403019571399",3,"-34.70480623405611,-56.04615274218908",103,22530
2,1,"-34.86347311448007,-56.17403019571399",4,"-32.412530637702794,-54.04713015664623",104,393686
3,1,"-34.86347311448007,-56.17403019571399",5,"-34.23198949912197,-57.74210900659811",105,172776
4,1,"-34.86347311448007,-56.17403019571399",6,"-33.29103587835989,-56.27098299307005",106,198970


In [42]:
# guarda
df_distancias_google.to_csv('tablas/df_distancias_centro_poblacion.csv', index=False)

00-fundamentacion.md	   imagenes
01-marco_teorico.md	   markdown
02-metodologia_files	   matriz_de_distancias_google-Copy1.ipynb
02-metodologia.ipynb	   matriz_distancias_centros_pobl.ipynb
02-metodologia.md	   notas.txt
capas			   README.md
centro_de_poblacion.ipynb  tablas
gantt_chart.ipynb
